# Do not change imports

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics.pairwise import cosine_similarity

from urllib.parse import urljoin
import itertools
import random

import matplotlib.pyplot as plt
%matplotlib inline

# A. MovieLens 100K Dataset: EDA via Pandas
TODO: Read the README (https://grouplens.org/datasets/movielens/100k/)

In [2]:
ML_100k_BASE_URL = 'http://files.grouplens.org/datasets/movielens/ml-100k/'

def movielens_url(fname):
    return urljoin(ML_100k_BASE_URL, fname)

## A.1. Read Genres

In [3]:
# names of the columns - see README, u.genre
genre_cols = ['genre', 'genre_id']

genres = pd.read_csv(movielens_url('u.genre'), sep='|', names=genre_cols)

genres

,genre,genre_id
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4
5,Comedy,5
6,Crime,6
7,Documentary,7
8,Drama,8
9,Fantasy,9


## A.2. Read Users

In [4]:
user_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

# TODO: read users from u.user
users = pd.read_csv(movielens_url('u.user'), sep='|', names=user_cols)

users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 5 columns):
user_id       943 non-null int64
age           943 non-null int64
sex           943 non-null object
occupation    943 non-null object
zip_code      943 non-null object
dtypes: int64(2), object(3)
memory usage: 36.9+ KB


## A.3. Read Movies

In [5]:
# Be lazy - let the genre table tell you the last set of column names
movie_cols = ['movie_id','movie_title','release_date','video_release_date','IMDb_URL'] + [g for g in genres.sort_values(by='genre_id')['genre']]

# TODO: read movies from u.item
movies = pd.read_csv(movielens_url('u.data'), sep='|', names=movie_cols)

movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
movie_id              100000 non-null object
movie_title           0 non-null float64
release_date          0 non-null float64
video_release_date    0 non-null float64
IMDb_URL              0 non-null float64
unknown               0 non-null float64
Action                0 non-null float64
Adventure             0 non-null float64
Animation             0 non-null float64
Children's            0 non-null float64
Comedy                0 non-null float64
Crime                 0 non-null float64
Documentary           0 non-null float64
Drama                 0 non-null float64
Fantasy               0 non-null float64
Film-Noir             0 non-null float64
Horror                0 non-null float64
Musical               0 non-null float64
Mystery               0 non-null float64
Romance               0 non-null float64
Sci-Fi                0 non-null float64
Thriller              0 n

In [6]:
# TODO: Interesting... find the row with a null release_date!

In [7]:
# TODO: assign movie_id (to check/clean up ratings later)
bad_row_id = None

# drop the row (in place!)
movies.drop(movies.loc[pd.isnull(movies.release_date)].index, inplace=True)

# TODO: confirm it's gone

In [8]:
# TODO: output a table including only the movies with null IMDB urls

In [9]:
# output all the fields of the first row
movies.loc[1357]

KeyError: 'the label [1357] is not in the [index]'

TODO: Try finding this movie with Google (it does exist) - what is special/different about it?

In [ ]:
# TODO: output all the fields of the second row

TODO: Try finding this movie with Google - comment on the likely name/origin of this movie

In [ ]:
# TODO: there is one movie with an unknown genre - what is it?

TODO: find this movie using Google - what is special about it? what would you indicate as its genre?

In [ ]:
# TODO: there is a field of the movies dataset that has no useful values - what is it?
bad_field = ''
movies.drop(bad_field, 1, inplace=True)

# It also turns out that the IMDB url's don't work :(
movies.drop('IMDb_URL', 1, inplace=True)

movies.head()

## A.4. Read Ratings

In [ ]:
rating_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

# TODO: read ratings from u.data
ratings = pd.read_csv(movielens_url('u.data'), sep='|', names=movie_cols)

ratings.info()

In [ ]:
# TODO: remember to drop ratings of our bad movie above

# TODO: confirm they are gone

## A.5. Merge Dataset

In [ ]:
lens = pd.merge(pd.merge(movies, ratings), users)
lens.head()

## A.6. Ask Interesting Questions

In [ ]:
# TODO: compute sparsity = 100% * (# ratings) / ((# users) * (# movies))
#       basically: what percentage of the full utility matrix is represented?

In [ ]:
# TODO: Aggregate by movie_id, get stats on number (i.e. size) of ratings per movie

In [ ]:
# TODO: Visualize the movie counts as a histogram

In [ ]:
# TODO: Aggregate by user_id, get stats on number (i.e. size) of ratings per user

In [ ]:
# TODO: Visualize the user counts as a histogram

TODO: comment on the last two stats tables/plots - what does this say in general about ratings per user/movie?

In [ ]:
# TODO: Which movie has the most ratings?

In [ ]:
# TODO: are all ratings equally likely? output a bar chart of rating frequency overall

In [ ]:
# TODO: of the movies that have 100 or more ratings, what are the top-ten ranked movies?

In [ ]:
# TODO: describe one more interesting question, and then query the result with Pandas

# B. Extract Utility Matrix

In [ ]:
def user_id_to_index(id):
    return id-1

def user_index_to_id(idx):
    return idx+1

# Accounts for the bad movie
def movie_id_to_index(id):
    return id-1 if id<bad_row_id else id-2

# Accounts for the bad movie
def movie_index_to_id(idx):
    return idx+1 if idx<(bad_row_id-1) else idx+2

In [ ]:
num_users = ratings.user_id.unique().shape[0]
num_movies = ratings.movie_id.unique().shape[0]

utility = np.zeros((num_users, num_movies))

for idx,rating in ratings.iterrows():
    utility[user_id_to_index(rating.user_id), movie_id_to_index(rating.movie_id)] = rating.rating

# should match above!
sparsity = float(len(utility.nonzero()[0]))
sparsity /= (utility.shape[0] * utility.shape[1])
sparsity *= 100
print('Sparsity: {:.2f}%'.format(sparsity))

# C. Evaluation via Mean Squared Error (MSE)

In [ ]:
a = [1, 2, 3, 4, 5]
b = [1, 1, 4, 2, 10]
print("{:.4f}".format(mse(a, b)))

TODO: Show the mathematical operations that reproduce the above result

$$ mse = \ldots $$

In [ ]:
def mse_utility(u1, u2):
    return mse(u1[u1.nonzero()].flatten(), u2[u2.nonzero()].flatten())

u1 = np.array([[2, 0, 0, 4, 4], [5, 5, 5, 3, 3], [2, 4, 2, 1, 2]])
u2 = np.array([[3, 0, 0, 4, 4], [5, 5, 5, 3, 3], [2, 4, 2, 1, 7]])

print("{:.4f}".format(mse_utility(u1, u2)))

TODO: 
1. Explain what operation is being performed. Include *why* it is appropriate in the context evaluating a recommender system, as well as performance implications.

2. Show the mathematical operations that reproduce the above result

$$ mse = \ldots $$

# D. Similarity via Cosine Distance

In [ ]:
# TODO: implement, only use primitive Python loops/operations
#       sim = (A . B) / ( ||A|| ||B|| )
def cosine_sim(v1, v2):
    return 0.0
    
v1 = [5, 0, 3, 0, 2, 0, 0, 2, 0, 0]
v2 = [3, 0, 2, 0, 1, 1, 0, 1, 0, 1]

print("Cosine Similarity: {:.6f}, expected={:.6f}".format(cosine_sim(v1, v2), cosine_similarity(np.array(v1).reshape(1, -1), np.array(v2).reshape(1, -1))[0][0]))

In [ ]:
def sim_matrix(u, eps=1.0e-9):
    step1 = u.dot(u.T) + eps
    step2 = np.array([np.sqrt(np.diagonal(step1))])
    return (step1 / step2 / step2.T)

print(sim_matrix(u1))
print(cosine_sim(u1[0], u1[1]))
print(cosine_sim(u1[0], u1[2]))
print(cosine_sim(u1[1], u1[2]))

In [ ]:
def sim_allpairs(u):
    n = u.shape[0]
    sim = np.eye(n)
    for x in itertools.combinations(range(n), 2):
        s = cosine_sim(u[x[0]], u[x[1]])
        sim[x[0], x[1]] = s
        sim[x[1], x[0]] = s
    
    return sim

print("MSE for U1: {:.4f}".format(mse_utility(sim_matrix(u1), sim_allpairs(u1))))
print("MSE for U2: {:.4f}".format(mse_utility(sim_matrix(u2), sim_allpairs(u2))))

In [ ]:
%timeit -n 10 -r 3 sim_matrix(utility[:50,:])

In [ ]:
%timeit -n 10 -r 3 sim_allpairs(utility[:50,:])

TODO:
1. Explain what each line of sim_matrix is doing (including why eps is necessary)
2. Explain why this approach is preferable to pairwise calls to cosine_sim (reference the previous two timeit calls; explain what each is doing and how it relates to your conclusion)

In [ ]:
def sim_users(u):
    return sim_matrix(u)

# TODO: use sim_matrix to return similarity between items (i.e. movies)
def sim_items(u):
    return None

print(sim_items(u1))
print(cosine_sim(u1[:, 0], u1[:, 1]))
print(cosine_sim(u1[:, 0], u1[:, 2]))
print(cosine_sim(u1[:, 0], u1[:, 3]))
print(cosine_sim(u1[:, 0], u1[:, 4]))
print(cosine_sim(u1[:, 1], u1[:, 2]))
print(cosine_sim(u1[:, 1], u1[:, 3]))
print(cosine_sim(u1[:, 1], u1[:, 4]))
print(cosine_sim(u1[:, 2], u1[:, 3]))
print(cosine_sim(u1[:, 2], u1[:, 4]))
print(cosine_sim(u1[:, 3], u1[:, 4]))

# E. K-Neighborhood

In [ ]:
# TODO: given a slice from a similarity matrix (arr),
#       index of "self" (self_idx),
#       and k
#       return dictionary of k most similar indexes as {idx:sim}
#       HINT: look at np.argsort
def top_k(arr, self_idx, k):
    return {}

In [ ]:
sim_u1 = sim_items(u1)

# Example via u1...
print(sim_u1)
print()

# Top-2 w.r.t. row 0 (should be {1: 0.897, 2:0.937})
print(sim_u1[0])
print(top_k(sim_u1[0], 0, 2))
print()

# Top-2 w.r.t. row 1 (should be {0: 0.897, 2:0.957})
print(sim_u1[1])
print(top_k(sim_u1[1], 1, 2))
print()

# Top-3 w.r.t. col 1 (should be {0: 0.897, 2:0.957, 4:0.667})
print(sim_u1[:, 1])
print(top_k(sim_u1[:, 1], 1, 3))
print()

# F. Recommend via Similar Users

In [ ]:
# TODO: given utility matrix (m_utility),
#       similarity of users (m_sim_users),
#       user index of interest (user_idx),
#       item index of interest (item_idx),
#       neighborhood size (k)
# 
#       output the average of the similarity-weighted ratings
#       of top-k similar users that have
#       rated the item
def rec_via_users(m_utility, m_sim_users, user_idx, item_idx, k):
    return 0
    
print(u1)
print()
print(sim_users(u1))
print()

# Expected: 5
print(rec_via_users(u1, sim_users(u1), 0, 1, 1))

# Expected: 4.54 ~ (.588 * 5) + (.495 * 4) / (.588 + .495)
print(rec_via_users(u1, sim_users(u1), 0, 1, 2))

## F.1. Evaluation

In [ ]:
random.seed(12345)

def recs_via_users(m_utility, m_sim_users, k, test_n):
    test = random.sample(range(m_sim_users.shape[0]), test_n)
    true = []
    pred = []
    for user_idx in test:
        for item_idx in range(m_utility.shape[1]):
            if m_utility[user_idx][item_idx] != 0:
                true.append(m_utility[user_idx][item_idx])
                
                p = round(rec_via_users(m_utility, m_sim_users, user_idx, item_idx, k))
                if p != 0:    
                    pred.append(p)
                else:
                    pred.append(1.0e-9)
                        

    return mse_utility(np.array([true], dtype=np.float64), np.array([pred], dtype=np.float64))
    
similarity_users = sim_users(utility)

ks = []
mses = []
for i in range(50):
    ks.append(i+1)
    mses.append(recs_via_users(utility, similarity_users, i+1, 100))
    print("{}/50".format(i+1), mses[-1])

In [ ]:
%timeit -n 10 -r 3 recs_via_users(utility, similarity_users, 2, 100)

In [ ]:
%timeit -n 10 -r 3 recs_via_users(utility, similarity_users, 20, 100)

In [ ]:
plt.plot(ks, mses)

TODO:
1. Describe what is being done in the recs_via_users function - what is the evaluation approach? What would have been a more "fair" methodology?
2. What do you notice happening as k increases?
3. Based upon these results, what is a reasonable neighborhood size?

# G. Recommend via SVD

In [ ]:
random.seed(12345)

# TODO: given a utility matrix and
#       number of singular values to use
# 
#       return a projected matrix
#       based upon a limited number of dimensions
#       U_pred = U S V^T
# 
#       HINT: the svds function does most of the work for you
def svd_projection(m_utility, num_dims):
    return None

def recs_via_svd(m_utility, num_dims, test_n):
    test = random.sample(range(m_utility.shape[0]), test_n)    
    svd_utility = svd_projection(utility, num_dims)
    
    true = []
    pred = []
    for user_idx in test:
        for item_idx in range(m_utility.shape[1]):
            if m_utility[user_idx][item_idx] != 0:
                true.append(m_utility[user_idx][item_idx])
                p = round(svd_utility[user_idx][item_idx])
                
                if p != 0:
                    pred.append(p)
                else:
                    pred.append(1.0e-9)

    return mse_utility(np.array([true], dtype=np.float64), np.array([pred], dtype=np.float64))
    
    
ds = []
mses = []
for i in range(100):
    ds.append(i+1)
    mses.append(recs_via_svd(utility, i+1, 100))
    print("{}/100".format(i+1), mses[-1])

In [ ]:
%timeit -n 10 -r 3 recs_via_svd(utility, 10, 100)

In [ ]:
%timeit -n 10 -r 3 recs_via_svd(utility, 20, 100)

In [ ]:
%timeit -n 10 -r 3 recs_via_svd(utility, 40, 100)

In [ ]:
%timeit -n 10 -r 3 recs_via_svd(utility, 80, 100)

In [ ]:
plt.plot(ds, mses)

TODO: comment on the relative speed/quality of the SVD approach as compared to the neighborhood approach

# H. TODO: Your Turn
The previous sections explored some building blocks and methods of recommendation for the MovieLens dataset.
In this section, you must extend this work:
1. Implement a method that was not covered above (e.g. item-based neighborhood, direct matrix factorization) OR extend a method (e.g. incorporate rating bias)
2. Evaluate your method similar to what was done above

Do NOT change any code above - only add code/comment in this section (as many cells as you see fit). You are welcome to copy-paste code from above.

# I. Extra Credit
If you so choose, create an actual recommendation engine using one of the supplied methods:
1. Create a function that takes as input a dictionary of {movie_title:rating}, as well as a k, and your function should return the top k movies that have not been watched and have highest predicted ratings.
2. Run the function on a few representative examples - analyze your results.